In [1]:
import os
import pandas as pd

# Create directory if it doesn't exist
os.makedirs("cleandata", exist_ok=True)

In [2]:
def clean_batting_data(raw_df):
    # Rename columns
    raw_df.columns = [col.strip().replace(" ", "_") for col in raw_df.columns]
    
    # Fix '90s' column (e.g., "1" → 1)
    if '90s' in raw_df.columns:
        raw_df['90s'] = pd.to_numeric(raw_df['90s'], errors='coerce').fillna(0)
    
    # Impute missing SR/Avg with T20 data
    raw_df['SR'] = raw_df['SR'].fillna(raw_df['Batting_T20_SR'])
    raw_df['Avg'] = raw_df['Avg'].fillna(raw_df['Batting_T20_Avg'])
    
    return raw_df

# Example usage
batting_stats = pd.read_csv("ipl_2024_25_stats\batting_stats\Most Runs.csv")
cleaned_batting = clean_batting_data(batting_stats)
cleaned_batting.to_csv("cleandata/batting_stats_clean.csv", index=False)

<>:16: SyntaxWarning: invalid escape sequence '\M'
<>:16: SyntaxWarning: invalid escape sequence '\M'
C:\Users\HP\AppData\Local\Temp\ipykernel_17964\4047533294.py:16: SyntaxWarning: invalid escape sequence '\M'
  batting_stats = pd.read_csv("ipl_2024_25_stats\batting_stats\Most Runs.csv")
C:\Users\HP\AppData\Local\Temp\ipykernel_17964\4047533294.py:16: SyntaxWarning: invalid escape sequence '\M'
  batting_stats = pd.read_csv("ipl_2024_25_stats\batting_stats\Most Runs.csv")


OSError: [Errno 22] Invalid argument: 'ipl_2024_25_stats\x08atting_stats\\Most Runs.csv'

In [12]:
# In data/cleaning.ipynb
import pandas as pd
import os

# Define paths
RAW_DATA_DIR = "data/raw"
CLEAN_DATA_DIR = "data/clean"
os.makedirs(CLEAN_DATA_DIR, exist_ok=True)
def clean_batting_data(file_path):
    df = pd.read_csv(f"{RAW_DATA_DIR}/{file_path}")
    

In [22]:
def clean_batting_data(file_path):
    df = pd.read_csv(f"{RAW_DATA_DIR}/{file_path}")
    
    # Rename columns
    df.columns = ['rank', 'player_name', 'team', 'matches', 'innings', 'runs', 
                 'average', 'strike_rate', 'fours', 'sixes', 'year']
    
    # Drop rank column (redundant)
    df.drop('rank', axis=1, inplace=True)
    
    # Convert numeric columns
    numeric_cols = ['matches', 'innings', 'runs', 'average', 'strike_rate', 'fours', 'sixes']
    df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors='coerce')
    
    # Add derived metrics
    df['boundary_pct'] = (df['fours'] + df['sixes']) / df['runs']
    df['consistency'] = df['average'] / df['strike_rate']
    
    # Standardize team names (e.g., "RCB" instead of "Royal Challengers Bengaluru")
    team_mapping = {'Royal Challengers Bengaluru': 'RCB', 'Chennai Super Kings': 'CSK'}  # Add all teams
    df['team'] = df['team'].replace(team_mapping)
    
    # Save cleaned data
    df.to_csv(f"{CLEAN_DATA_DIR}/cleaned_batting.csv", index=False)
    return df

batting_clean = clean_batting_data("most_runs.csv")
batting_clean.head()

,player_name,team,matches,innings,runs,average,strike_rate,fours,sixes,year,boundary_pct,consistency
0,Virat Kohli,15,15,741,61.75,154.70,62,38,2024,NaN,33.392713,2.495161
1,Ruturaj Gaikwad,14,14,583,53.00,141.16,58,18,2024,NaN,38.528302,2.433793
2,Riyan Parag,15,14,573,52.09,149.22,40,33,2024,NaN,39.489345,3.730500
3,Travis Head,15,15,567,40.50,191.55,64,32,2024,NaN,50.765432,2.992969
4,Sanju Samson,15,15,531,48.27,153.47,48,24,2024,NaN,42.428009,3.197292


In [26]:

df = pd.read_csv(r'data\raw\most_runs.csv')
    

In [23]:
df.columns = ['rank', 'player_name', 'team', 'matches', 'innings', 'runs', 
              'average', 'strike_rate', 'fours', 'sixes', 'year']


In [28]:
expected_cols = ['rank', 'player_name', 'team', 'matches', 'innings', 'runs', 
                 'average', 'strike_rate', 'fours', 'sixes', 'year']

# Only assign names if the length matches
if len(df.columns) == len(expected_cols):
    df.columns = expected_cols
else:
    print(f"Unexpected number of columns: {len(df.columns)}")
    print("Columns are:", df.columns)


In [29]:
import pandas as pd
import numpy as np
import os

# Define directories (adjust these if needed)
RAW_DATA_DIR = "data/raw"
CLEAN_DATA_DIR = "data/cleaned"

# Create cleaned directory if it doesn't exist
os.makedirs(CLEAN_DATA_DIR, exist_ok=True)

def clean_batting_data(file_path):
    # Read CSV
    df = pd.read_csv(f"{RAW_DATA_DIR}/{file_path}")
    
    # Print original columns for debugging
    print("Original columns:", df.columns.tolist())
    
    # Expected column names (adjust if your CSV structure is different)
    expected_cols = ['rank', 'player_name', 'team', 'matches', 'innings', 'runs', 
                     'average', 'strike_rate', 'fours', 'sixes', 'year']
    
    if len(df.columns) == len(expected_cols):
        df.columns = expected_cols
    elif len(df.columns) == len(expected_cols) - 1:
        # Assume 'year' column is missing
        df.columns = expected_cols[:-1]
        df['year'] = np.nan  # Add year as NaN
    else:
        raise ValueError(f"Unexpected number of columns: {len(df.columns)}")

    # Drop the 'rank' column (not needed)
    df.drop('rank', axis=1, inplace=True)

    # Convert columns to numeric where appropriate
    numeric_cols = ['matches', 'innings', 'runs', 'average', 'strike_rate', 'fours', 'sixes']
    df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors='coerce')

    # Avoid division by zero in derived metrics
    df['runs'].replace(0, np.nan, inplace=True)
    df['strike_rate'].replace(0, np.nan, inplace=True)

    # Add derived metrics
    df['boundary_pct'] = (df['fours'] + df['sixes']) / df['runs']
    df['consistency'] = df['average'] / df['strike_rate']

    # Standardize team names
    team_mapping = {
        'Royal Challengers Bengaluru': 'RCB',
        'Chennai Super Kings': 'CSK',
        'Mumbai Indians': 'MI',
        'Kolkata Knight Riders': 'KKR',
        'Sunrisers Hyderabad': 'SRH',
        'Delhi Capitals': 'DC',
        'Punjab Kings': 'PBKS',
        'Rajasthan Royals': 'RR',
        'Gujarat Titans': 'GT',
        'Lucknow Super Giants': 'LSG'
    }
    df['team'] = df['team'].replace(team_mapping)

    # Save cleaned data
    df.to_csv(f"{CLEAN_DATA_DIR}/cleaned_batting.csv", index=False)
    print("✅ Cleaned data saved successfully!")

    return df

# Run the function
batting_clean = clean_batting_data("most_runs.csv")

# Preview cleaned data
print(batting_clean.head())


Original columns: ['rank', 'Player', 'Team', 'Matches', 'Inns', 'Runs', 'Avg', 'SR', '4s', '6s', 'Year']
✅ Cleaned data saved successfully!
       player_name  team  matches  innings   runs  average  strike_rate  \
0      Virat Kohli    15       15      741  61.75   154.70           62   
1  Ruturaj Gaikwad    14       14      583  53.00   141.16           58   
2      Riyan Parag    15       14      573  52.09   149.22           40   
3      Travis Head    15       15      567  40.50   191.55           64   
4     Sanju Samson    15       15      531  48.27   153.47           48   

   fours  sixes  year  boundary_pct  consistency  
0     38   2024   NaN     33.392713     2.495161  
1     18   2024   NaN     38.528302     2.433793  
2     33   2024   NaN     39.489345     3.730500  
3     32   2024   NaN     50.765432     2.992969  
4     24   2024   NaN     42.428009     3.197292  


C:\Users\HP\AppData\Local\Temp\ipykernel_17964\1116026494.py:40: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['runs'].replace(0, np.nan, inplace=True)
C:\Users\HP\AppData\Local\Temp\ipykernel_17964\1116026494.py:41: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, whe

In [32]:
def clean_bowling_data(file_path):
    df = pd.read_csv(f"{RAW_DATA_DIR}/{file_path}")
    
    # Rename columns
    df.columns = ['rank', 'player_name', 'team', 'matches', 'overs', 'wickets', 
                 'average', 'economy', 'strike_rate', 'year']
    
    # Clean BBI column if exists
    if 'bbi' in df.columns:
        df['bbi'] = df['bbi'].str.replace("May-", "5/").str.replace("Apr-", "4/")
    
    # Convert numeric columns
    numeric_cols = ['matches', 'overs', 'wickets', 'average', 'economy', 'strike_rate']
    df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors='coerce')
    
    # Add derived metrics
    df['wickets_per_match'] = df['wickets'] / df['matches']
    
    # Save cleaned data
    df.to_csv(f"{CLEAN_DATA_DIR}/cleaned_bowling.csv", index=False)
    return df

bowling_clean = clean_bowling_data("most_wickets.csv")
bowling_clean.head()

ValueError: Length mismatch: Expected axis has 11 elements, new values have 10 elements

In [33]:
import pandas as pd
import numpy as np
import os

def clean_bowling_data(file_path):
    # Read data with proper path handling (use raw string or forward slashes)
    try:
        df = pd.read_csv(os.path.join(RAW_DATA_DIR, file_path))
    except FileNotFoundError:
        print(f"Error: File not found at {os.path.join(RAW_DATA_DIR, file_path)}")
        return None
    
    # Check current columns
    print("Original columns:", df.columns.tolist())
    
    # Expected columns in your data (adjust based on actual CSV)
    expected_columns = ['rank', 'Player', 'Team', 'Matches', 'Overs', 'Wkts', 
                       'Avg', 'Econ', 'SR', 'BBI', 'Year']
    
    # Verify column count matches
    if len(df.columns) != len(expected_columns):
        print(f"⚠️ Warning: Expected {len(expected_columns)} columns, found {len(df.columns)}")
        print("Actual columns:", df.columns.tolist())
        return None
    
    # Rename columns
    df.columns = expected_columns
    
    # Clean BBI column
    if 'BBI' in df.columns:
        df['BBI'] = df['BBI'].astype(str).str.replace("May-", "5/").str.replace("Apr-", "4/")
    
    # Convert numeric columns (safer method)
    numeric_cols = ['Matches', 'Overs', 'Wkts', 'Avg', 'Econ', 'SR']
    for col in numeric_cols:
        df[col] = pd.to_numeric(df[col], errors='coerce')
    
    # Calculate derived metrics (avoid inplace warnings)
    df = df.assign(
        wickets_per_match=df['Wkts'] / df['Matches'],
        bowling_impact=df['Wkts'] * (1/df['Econ'])
    )
    
    # Handle zeros/missing values properly
    df['Avg'] = df['Avg'].replace(0, np.nan)
    df['SR'] = df['SR'].replace(0, np.nan)
    
    # Save cleaned data
    output_path = os.path.join(CLEAN_DATA_DIR, "cleaned_bowling.csv")
    df.to_csv(output_path, index=False)
    print(f"✅ Cleaned data saved to {output_path}")
    return df

# Usage:
RAW_DATA_DIR = "data/raw"  # or r"data\raw" for Windows
CLEAN_DATA_DIR = "data/clean"
os.makedirs(CLEAN_DATA_DIR, exist_ok=True)

bowling_clean = clean_bowling_data("most_wickets.csv")
if bowling_clean is not None:
    display(bowling_clean.head())

Original columns: ['rank', 'Player', 'Matches', 'Overs', 'Balls', 'Wkts', 'Avg', 'Runs', '4-fers', '5-fers', 'Year']
✅ Cleaned data saved to data/clean\cleaned_bowling.csv


,rank,Player,Team,Matches,Overs,Wkts,Avg,Econ,SR,BBI,Year,wickets_per_match,bowling_impact
0,1,Harshal Patel,14,49.0,294,24,19.88,477,NaN,-,2024,0.489796,0.050314
1,2,Varun Chakaravarthy,14,50.0,300,21,19.14,402,NaN,-,2024,0.420000,0.052239
2,3,Jasprit Bumrah,13,51.5,311,20,16.80,336,NaN,1,2024,0.388350,0.059524
3,4,T Natarajan,14,51.2,308,19,24.47,465,1.0,-,2024,0.371094,0.040860
4,5,Harshit Rana,12,42.1,253,19,20.16,383,NaN,-,2024,0.451306,0.049608


In [34]:
import pandas as pd
import numpy as np
import os
from pathlib import Path

# Setup paths
BASE_DIR = Path.cwd()
RAW_DATA_DIR = BASE_DIR / "data" / "raw"
CLEAN_DATA_DIR = BASE_DIR / "data" / "clean"
os.makedirs(CLEAN_DATA_DIR, exist_ok=True)

def clean_batting_data(file_name="most_runs.csv"):
    """Clean batting stats data"""
    try:
        df = pd.read_csv(RAW_DATA_DIR / "batting_stats" / file_name)
        
        # Rename columns
        df.columns = ['rank', 'player_name', 'team', 'matches', 'innings', 
                     'runs', 'average', 'strike_rate', 'fours', 'sixes', 'year']
        
        # Convert and clean data
        numeric_cols = ['matches', 'innings', 'runs', 'average', 'strike_rate', 'fours', 'sixes']
        df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors='coerce')
        
        # Calculate metrics
        df['boundary_pct'] = (df['fours'] + df['sixes']) / df['runs']
        df['consistency'] = df['average'] / df['strike_rate']
        
        # Save cleaned data
        output_path = CLEAN_DATA_DIR / "cleaned_batting.csv"
        df.to_csv(output_path, index=False)
        print(f"✅ Batting data cleaned and saved to {output_path}")
        return df
        
    except Exception as e:
        print(f"❌ Error cleaning batting data: {str(e)}")
        return None

def clean_bowling_data(file_name="most_wickets.csv"):
    """Clean bowling stats data"""
    try:
        df = pd.read_csv(RAW_DATA_DIR / "bowling_stats" / file_name)
        
        # Rename columns
        df.columns = ['rank', 'player_name', 'matches', 'overs', 'balls', 
                     'wickets', 'average', 'runs', '4_fers', '5_fers', 'year']
        
        # Convert and clean data
        numeric_cols = ['matches', 'overs', 'balls', 'wickets', 'average', 'runs']
        df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors='coerce')
        
        # Calculate metrics
        df['wickets_per_match'] = df['wickets'] / df['matches']
        df['economy_rate'] = df['runs'] / df['overs']
        df['bowling_impact'] = df['wickets'] * (1/df['economy_rate'])
        
        # Save cleaned data
        output_path = CLEAN_DATA_DIR / "cleaned_bowling.csv"
        df.to_csv(output_path, index=False)
        print(f"✅ Bowling data cleaned and saved to {output_path}")
        return df
        
    except Exception as e:
        print(f"❌ Error cleaning bowling data: {str(e)}")
        return None

def clean_venue_data(file_name="historical_pitch_data.csv"):
    """Clean venue/pitch data"""
    try:
        df = pd.read_csv(RAW_DATA_DIR / file_name)
        
        # Clean and extract pitch metrics
        df['pace_avg'] = df['pitch_report'].str.extract(r'Pace Bowling\n(\d+\.?\d*)').astype(float)
        df['spin_avg'] = df['pitch_report'].str.extract(r'Spin Bowling\n(\d+\.?\d*)').astype(float)
        df['pitch_type'] = np.where(df['spin_avg'] > df['pace_avg'], 'spin', 'pace')
        
        # Save cleaned data
        output_path = CLEAN_DATA_DIR / "cleaned_venues.csv"
        df.to_csv(output_path, index=False)
        print(f"✅ Venue data cleaned and saved to {output_path}")
        return df
        
    except Exception as e:
        print(f"❌ Error cleaning venue data: {str(e)}")
        return None

def clean_matchup_data(file_name="hot_batters.csv"):
    """Clean player matchup data"""
    try:
        df = pd.read_csv(RAW_DATA_DIR / file_name)
        
        # Standardize column names
        df.columns = [col.lower().replace(' ', '_') for col in df.columns]
        
        # Clean strike rate
        if 'sr' in df.columns:
            df['sr'] = pd.to_numeric(df['sr'].str.rstrip('%'), errors='coerce')
        
        # Save cleaned data
        output_path = CLEAN_DATA_DIR / "cleaned_matchups.csv"
        df.to_csv(output_path, index=False)
        print(f"✅ Matchup data cleaned and saved to {output_path}")
        return df
        
    except Exception as e:
        print(f"❌ Error cleaning matchup data: {str(e)}")
        return None

def clean_all_data():
    """Run all cleaning functions"""
    print("Starting data cleaning process...\n")
    
    batting = clean_batting_data()
    bowling = clean_bowling_data()
    venues = clean_venue_data()
    matchups = clean_matchup_data()
    
    print("\nData cleaning complete!")
    return batting, bowling, venues, matchups

# Execute all cleaners
clean_all_data()

Starting data cleaning process...

❌ Error cleaning batting data: [Errno 2] No such file or directory: 'c:\\Users\\HP\\Downloads\\data\\data\\raw\\batting_stats\\most_runs.csv'
❌ Error cleaning bowling data: [Errno 2] No such file or directory: 'c:\\Users\\HP\\Downloads\\data\\data\\raw\\bowling_stats\\most_wickets.csv'
❌ Error cleaning venue data: 'pitch_report'
❌ Error cleaning matchup data: [Errno 2] No such file or directory: 'c:\\Users\\HP\\Downloads\\data\\data\\raw\\hot_batters.csv'

Data cleaning complete!


(None, None, None, None)

In [35]:
import pandas as pd
import numpy as np
import os
from pathlib import Path

# Setup paths - ADJUST THIS TO YOUR ACTUAL DATA LOCATION
BASE_DIR = Path.cwd()
RAW_DATA_DIR = BASE_DIR / "ipl_2024_25_stats"  # Changed to match your structure
CLEAN_DATA_DIR = BASE_DIR / "cleandata"
os.makedirs(CLEAN_DATA_DIR, exist_ok=True)

def clean_batting_data(file_name="Most Runs.csv"):
    """Clean batting stats data"""
    try:
        file_path = RAW_DATA_DIR / "batting_stats" / file_name
        if not file_path.exists():
            print(f"⚠️ File not found: {file_path}")
            return None
            
        df = pd.read_csv(file_path)
        print("Original batting columns:", df.columns.tolist())
        
        # Rename columns (adjust based on your actual columns)
        column_map = {
            'rank': 'rank',
            'Player': 'player_name',
            'Team': 'team',
            'Matches': 'matches',
            'Inns': 'innings',
            'Runs': 'runs',
            'Avg': 'average',
            'SR': 'strike_rate',
            '4s': 'fours',
            '6s': 'sixes',
            'Year': 'year'
        }
        df = df.rename(columns=column_map)
        
        # Convert numeric columns
        numeric_cols = ['matches', 'innings', 'runs', 'average', 'strike_rate', 'fours', 'sixes']
        for col in numeric_cols:
            if col in df.columns:
                df[col] = pd.to_numeric(df[col], errors='coerce')
        
        # Calculate metrics
        if all(col in df.columns for col in ['runs', 'fours', 'sixes']):
            df['boundary_pct'] = (df['fours'] + df['sixes']) / df['runs']
        if all(col in df.columns for col in ['average', 'strike_rate']):
            df['consistency'] = df['average'] / df['strike_rate']
        
        # Save cleaned data
        output_path = CLEAN_DATA_DIR / "cleaned_batting.csv"
        df.to_csv(output_path, index=False)
        print(f"✅ Batting data saved to {output_path}")
        return df
        
    except Exception as e:
        print(f"❌ Error cleaning batting data: {str(e)}")
        return None

def clean_bowling_data(file_name="Most Wickets.csv"):
    """Clean bowling stats data"""
    try:
        file_path = RAW_DATA_DIR / "bowling_stats" / file_name
        if not file_path.exists():
            print(f"⚠️ File not found: {file_path}")
            return None
            
        df = pd.read_csv(file_path)
        print("Original bowling columns:", df.columns.tolist())
        
        # Rename columns (adjust based on your actual columns)
        column_map = {
            'rank': 'rank',
            'Player': 'player_name',
            'Matches': 'matches',
            'Overs': 'overs',
            'Balls': 'balls',
            'Wkts': 'wickets',
            'Avg': 'average',
            'Runs': 'runs_conceded',
            '4-fers': '4_fers',
            '5-fers': '5_fers',
            'Year': 'year'
        }
        df = df.rename(columns=column_map)
        
        # Convert numeric columns
        numeric_cols = ['matches', 'overs', 'balls', 'wickets', 'average', 'runs_conceded']
        for col in numeric_cols:
            if col in df.columns:
                df[col] = pd.to_numeric(df[col], errors='coerce')
        
        # Calculate metrics
        if all(col in df.columns for col in ['wickets', 'matches']):
            df['wickets_per_match'] = df['wickets'] / df['matches']
        if all(col in df.columns for col in ['runs_conceded', 'overs']):
            df['economy_rate'] = df['runs_conceded'] / df['overs']
            df['bowling_impact'] = df['wickets'] * (1/df['economy_rate'])
        
        # Save cleaned data
        output_path = CLEAN_DATA_DIR / "cleaned_bowling.csv"
        df.to_csv(output_path, index=False)
        print(f"✅ Bowling data saved to {output_path}")
        return df
        
    except Exception as e:
        print(f"❌ Error cleaning bowling data: {str(e)}")
        return None

def clean_all_data():
    """Run all cleaning functions"""
    print("Starting data cleaning process...\n")
    
    # First verify directory structure
    print(f"Looking for data in: {RAW_DATA_DIR}")
    if not RAW_DATA_DIR.exists():
        print(f"❌ Error: Data directory not found at {RAW_DATA_DIR}")
        print("Please adjust the BASE_DIR or RAW_DATA_DIR variables")
        return None
    
    batting = clean_batting_data()
    bowling = clean_bowling_data()
    
    print("\nData cleaning complete!")
    return batting, bowling

# Execute all cleaners
cleaned_data = clean_all_data()

Starting data cleaning process...

Looking for data in: c:\Users\HP\Downloads\data\ipl_2024_25_stats
Original batting columns: ['Player', 'Team', 'Matches', 'Inns', 'Runs', 'Avg', 'SR', '4s', '6s', 'Year']
✅ Batting data saved to c:\Users\HP\Downloads\data\cleandata\cleaned_batting.csv
⚠️ File not found: c:\Users\HP\Downloads\data\ipl_2024_25_stats\bowling_stats\Most Wickets.csv

Data cleaning complete!


In [38]:
from pathlib import Path

def list_files(path, indent=0):
    for p in sorted(Path(path).iterdir()):
        print("    " * indent + ("📁 " if p.is_dir() else "📄 ") + p.name)
        if p.is_dir():
            list_files(p, indent + 1)

list_files("Data")


FileNotFoundError: [WinError 3] The system cannot find the path specified: 'Data'

In [40]:
from pathlib import Path

def list_files(path, indent=0):
    try:
        for p in sorted(Path(path).iterdir()):
            print("    " * indent + ("📁 " if p.is_dir() else "📄 ") + p.name)
            if p.is_dir():
                list_files(p, indent + 1)
    except FileNotFoundError:
        print(f"❌ Path not found: {path}")

# Use the correct path (with forward slashes)
list_files("C:/Users/HP/Downloads/data")


📄 cleaning.ipynb
📄 historical_pitch_data.csv
📁 ipl_2024_25_stats
    📁 batting_stats
        📄 Best Batting Average.csv
        📄 Best Batting Strike Rate.csv
        📄 Highest Scores.csv
        📄 Most Fifties.csv
        📄 Most Fours.csv
        📄 Most Hundreds.csv
        📄 Most Nineties.csv
        📄 Most Runs.csv
        📄 Most Sixes.csv
    📁 bowlling_stats
        📄 Best Bowling Average.csv
        📄 Best Bowling Strike Rate.csv
        📄 Best Bowling.csv
        📄 Best Economy.csv
        📄 Most 5 Wickets.csv
        📄 Most Wickets.csv
📄 ipl_bowling_stats (3).csv
📄 ipl_partnerships_last10years.csv
📁 matchup
    📄 cold_batters.csv
    📄 cold_bowlers.csv
    📄 hot-batters.csv
    📄 hot_bowlers.csv
📄 partnership.py
📁 t20_ipl_data
    📄 player_perfomance.csv
📁 venue_match
    📄 venu_match_detail.csv


In [41]:
import pandas as pd
import numpy as np
from pathlib import Path
import os

# Setup paths
BASE_DIR = Path.cwd()
DATA_DIR = BASE_DIR / "Data"
CLEAN_DATA_DIR = DATA_DIR / "clean_data"
os.makedirs(CLEAN_DATA_DIR, exist_ok=True)

def clean_batting_stats():
    """Clean all batting statistics CSVs"""
    batting_stats_dir = DATA_DIR / "ipl_2024_25_stats" / "batting_stats"
    cleaned_files = {}
    
    for file in batting_stats_dir.glob("*.csv"):
        try:
            df = pd.read_csv(file)
            file_name = file.stem
            
            # Common cleaning for all batting files
            df.columns = [col.strip().replace(" ", "_").lower() for col in df.columns]
            
            # Special handling for different batting files
            if "most_runs" in file_name:
                df = df.rename(columns={
                    'inns': 'innings',
                    'avg': 'average',
                    'sr': 'strike_rate'
                })
                df['boundary_pct'] = (df['4s'] + df['6s']) / df['runs']
                
            elif "highest_scores" in file_name:
                df = df.rename(columns={
                    'sr': 'strike_rate',
                    'vs': 'opponent'
                })
                
            # Save cleaned file
            clean_path = CLEAN_DATA_DIR / f"cleaned_{file_name}.csv"
            df.to_csv(clean_path, index=False)
            cleaned_files[file_name] = clean_path
            
        except Exception as e:
            print(f"❌ Error cleaning {file.name}: {str(e)}")
    
    return cleaned_files

def clean_bowling_stats():
    """Clean all bowling statistics CSVs"""
    bowling_stats_dir = DATA_DIR / "ipl_2024_25_stats" / "bowlling_stats"  # Note: "bowlling" spelling
    cleaned_files = {}
    
    for file in bowling_stats_dir.glob("*.csv"):
        try:
            df = pd.read_csv(file)
            file_name = file.stem
            
            # Common cleaning for all bowling files
            df.columns = [col.strip().replace(" ", "_").lower() for col in df.columns]
            
            # Special handling for different bowling files
            if "most_wickets" in file_name:
                df = df.rename(columns={
                    'wkts': 'wickets',
                    'econ': 'economy',
                    'sr': 'strike_rate'
                })
                df['wickets_per_match'] = df['wickets'] / df['matches']
                
            elif "best_bowling" in file_name:
                df['bbi'] = df['bbi'].str.replace("May-", "5/").str.replace("Apr-", "4/")
                
            # Save cleaned file
            clean_path = CLEAN_DATA_DIR / f"cleaned_{file_name}.csv"
            df.to_csv(clean_path, index=False)
            cleaned_files[file_name] = clean_path
            
        except Exception as e:
            print(f"❌ Error cleaning {file.name}: {str(e)}")
    
    return cleaned_files

def clean_other_files():
    """Clean remaining CSV files"""
    other_files = {
        "historical_pitch_data.csv": {
            "clean_ops": lambda df: df.assign(
                pitch_type=np.where(df['spin_wickets'] > df['pace_wickets'], 'spin', 'pace')
            )
        },
        "player_performance.csv": {
            "path": DATA_DIR / "t20_ipl_data" / "player_perfomance.csv",  # Note: "perfomance" spelling
            "clean_ops": lambda df: df.fillna(0)
        },
        "venue_match_details.csv": {
            "path": DATA_DIR / "venue_match" / "venu_match_detail.csv",  # Note: "venu" spelling
            "clean_ops": lambda df: df.rename(columns={
                '1st_inn_avg_runs': 'first_inn_avg_runs',
                '2nd_inn_avg_runs': 'second_inn_avg_runs'
            })
        }
    }
    
    cleaned_files = {}
    
    for file_name, config in other_files.items():
        try:
            path = config.get("path", DATA_DIR / file_name)
            df = pd.read_csv(path)
            
            # Apply cleaning operations
            df = config["clean_ops"](df)
            
            # Standardize column names
            df.columns = [col.strip().replace(" ", "_").lower() for col in df.columns]
            
            # Save cleaned file
            clean_path = CLEAN_DATA_DIR / f"cleaned_{file_name}"
            df.to_csv(clean_path, index=False)
            cleaned_files[file_name] = clean_path
            
        except Exception as e:
            print(f"❌ Error cleaning {file_name}: {str(e)}")
    
    return cleaned_files

def clean_all_data():
    """Run all cleaning functions"""
    print("Starting data cleaning process...\n")
    
    print("Cleaning batting stats...")
    batting_results = clean_batting_stats()
    
    print("\nCleaning bowling stats...")
    bowling_results = clean_bowling_stats()
    
    print("\nCleaning other files...")
    other_results = clean_other_files()
    
    print("\nData cleaning complete!")
    return {
        "batting": batting_results,
        "bowling": bowling_results,
        "other": other_results
    }

# Execute all cleaners
cleaning_report = clean_all_data()

Starting data cleaning process...

Cleaning batting stats...

Cleaning bowling stats...

Cleaning other files...
❌ Error cleaning historical_pitch_data.csv: [Errno 2] No such file or directory: 'c:\\Users\\HP\\Downloads\\data\\Data\\historical_pitch_data.csv'
❌ Error cleaning player_performance.csv: [Errno 2] No such file or directory: 'c:\\Users\\HP\\Downloads\\data\\Data\\t20_ipl_data\\player_perfomance.csv'
❌ Error cleaning venue_match_details.csv: [Errno 2] No such file or directory: 'c:\\Users\\HP\\Downloads\\data\\Data\\venue_match\\venu_match_detail.csv'

Data cleaning complete!


In [ ]:
import os
import csv

def print_directory_structure(root_dir, indent=""):
    for item in os.listdir(root_dir):
        path = os.path.join(root_dir, item)
        if os.path.isdir(path):
            print(f"{indent}📁 {item}/")
            print_directory_structure(path, indent + "    ")
        else:
            print(f"{indent}📄 {item}")
            if item.lower().endswith(".csv"):
                try:
                    with open(path, mode='r', encoding='utf-8') as csv_file:
                        reader = csv.reader(csv_file)
                        header = next(reader, None)
                        if header:
                            print(f"{indent}    🧾 CSV Header: {header}")
                        else:
                            print(f"{indent}    ⚠️ Empty CSV or no header")
                except Exception as e:
                    print(f"{indent}    ❌ Error reading CSV: {e}")

# Main directory to scan
main_directory = "dAT"
print(f"\n📂 Directory structure of: {main_directory}\n")
print_directory_structure(main_directory)



📂 Directory structure of: IPL_MODEL



FileNotFoundError: [WinError 3] The system cannot find the path specified: 'IPL_MODEL'

In [2]:
import os
import csv

def print_directory_structure(root_dir, indent=""):
    for item in os.listdir(root_dir):
        path = os.path.join(root_dir, item)
        if os.path.isdir(path):
            print(f"{indent}📁 {item}/")
            print_directory_structure(path, indent + "    ")
        else:
            print(f"{indent}📄 {item}")
            if item.lower().endswith(".csv"):
                try:
                    with open(path, mode='r', encoding='utf-8') as csv_file:
                        reader = csv.reader(csv_file)
                        header = next(reader, None)
                        if header:
                            print(f"{indent}    🧾 CSV Header: {header}")
                        else:
                            print(f"{indent}    ⚠️ Empty CSV or no header")
                except Exception as e:
                    print(f"{indent}    ❌ Error reading CSV: {e}")

# Main directory to scan
main_directory = "DATA"
print(f"\n📂 Directory structure of: {main_directory}\n")
print_directory_structure(main_directory)



📂 Directory structure of: DATA

📄 batter_vs_bowler_summary.csv
    🧾 CSV Header: ['batter', 'bowler', 'total_balls', 'total_runs', 'dot_balls', 'fours', 'sixes', 'boundaries', 'dismissals', 'extras_conceded', 'strike_rate', 'boundary_percentage', 'dot_percentage', 'runs_per_dismissal']
📁 clean_data/
    📄 cleaned_Best_Bowling.csv
        🧾 CSV Header: ['unnamed:_0', 'bowler', 'vs', 'ovs', 'runs', 'wkts', 'bbi', 'maidens', 'econ', 'year', 'bbi']
    📄 cleaned_Highest_Scores.csv
        🧾 CSV Header: ['unnamed:_0', 'batter', 'runs', 'balls', 'sr', 'opponent', '4s', '6s', 'years']
    📄 cleaned_historical_pitch_data.csv
        🧾 CSV Header: ['venue', 'first_innings_score', 'second_innings_score', 'toss_winner', 'winner', 'toss_decision', 'first_innings_boundaries', 'first_innings_sixes', 'second_innings_boundaries', 'second_innings_sixes', 'spin_wickets', 'pace_wickets', 'pitch_type']
    📄 cleaned_Most_Runs.csv
        🧾 CSV Header: ['player', 'team', 'matches', 'inns', 'runs', 'avg', 

In [3]:
import os

def print_directory_structure(root_dir, indent=""):
    for item in os.listdir(root_dir):
        path = os.path.join(root_dir, item)
        if os.path.isdir(path):
            print(f"{indent}📁 {item}/")
            print_directory_structure(path, indent + "    ")
        else:
            print(f"{indent}📄 {item}")

# Path to your nested directory
main_directory = os.path.join("IPL_MODEL", "DATA")
print(f"\n📂 Directory structure of: {main_directory}\n")
print_directory_structure(main_directory)



📂 Directory structure of: IPL_MODEL\DATA



FileNotFoundError: [WinError 3] The system cannot find the path specified: 'IPL_MODEL\\DATA'

In [1]:
import os

def print_top_level_structure(root_dir):
    print(f"\n📂 Top-level structure of: {root_dir}\n")
    for item in os.listdir(root_dir):
        path = os.path.join(root_dir, item)
        if os.path.isdir(path):
            print(f"📁 {item}/")
        else:
            print(f"📄 {item}")

# Use full absolute path
main_directory = r"C:\Users\HP\Downloads\ipl_model"
print_top_level_structure(main_directory)



📂 Top-level structure of: C:\Users\HP\Downloads\ipl_model

📄 chromedriver.exe
📄 cleaner.py
📄 cleaning.ipynb
📄 clean_data.py
📄 clean_training_data.csv
📁 Data/
📄 fantasy_model.pkl
📄 feature_engineer.py
📄 fixed_team_mapping.py
📄 ipl_2025_full_schedule.csv
📄 ipl_schedule.py
📄 live_data.py
📄 merged_training_data.csv
📄 merge_data.py
📄 model_training.py
📄 partnership.py


In [1]:
# partnership_data_cleaner.ipynb
import pandas as pd
from pathlib import Path
import re

# Configuration
INPUT_CSV = "Data/ipl_partnerships_last10years.csv"  # Your current file without headers
OUTPUT_CSV = "Data/partnerships_clean.csv"  # Cleaned output file

# 1. Load the raw data (no headers)
try:
    df = pd.read_csv(INPUT_CSV, header=None)
    print(f"Raw data loaded with {len(df)} rows")
except FileNotFoundError:
    print(f"Error: File not found at {INPUT_CSV}")
    raise

# 2. Assign temporary column names
temp_columns = [
    'players_pair',
    'runs',
    'innings',
    'team',
    'versus',
    'venue',
    'date',
    'match_type'
]
df.columns = temp_columns[:len(df.columns)]  # Handle files with varying columns

# 3. Clean and separate player names
def split_players(pair_str):
    """Split 'Player1, Player2' into separate names"""
    try:
        # Remove quotes and split on comma (but not within names)
        players = re.split(r',\s*(?![^()]*\))', pair_str.strip('"'))
        return [p.strip() for p in players]
    except:
        return [None, None]

df[['player1', 'player2']] = df['players_pair'].apply(
    lambda x: pd.Series(split_players(x))
)

# 4. Clean numeric fields
df['runs'] = df['runs'].str.replace('*', '').astype(int)  # Remove not-outs
df['innings'] = df['innings'].str.extract(r'(\d+)').astype(int)

# 5. Standardize team names
team_mapping = {
    'RCB': 'Royal Challengers Bangalore',
    'Guj Lions': 'Gujarat Lions',
    'MI': 'Mumbai Indians',
    'KKR': 'Kolkata Knight Riders',
    'GT': 'Gujarat Titans',
    'CSK': 'Chennai Super Kings',
    'Kings XI': 'Punjab Kings',
    'Daredevils': 'Delhi Capitals',
    'Chargers': 'Deccan Chargers'
}

df['team'] = df['team'].map(team_mapping).fillna(df['team'])
df['versus'] = df['versus'].str.replace('v ', '').map(team_mapping).fillna(df['versus'])

# 6. Clean dates
df['date'] = pd.to_datetime(df['date'], errors='coerce').dt.strftime('%Y-%m-%d')

# 7. Select and reorder final columns
final_columns = [
    'player1',
    'player2',
    'runs',
    'innings',
    'team',
    'versus',
    'venue',
    'date',
    'match_type'
]

clean_df = df[final_columns].copy()

# 8. Save cleaned data
clean_df.to_csv(OUTPUT_CSV, index=False)
print(f"Cleaned data saved to {OUTPUT_CSV} with columns: {clean_df.columns.tolist()}")

# 9. Display sample
clean_df.head()

Raw data loaded with 100 rows
Cleaned data saved to Data/partnerships_clean.csv with columns: ['player1', 'player2', 'runs', 'innings', 'team', 'versus', 'venue', 'date', 'match_type']


,player1,player2,runs,innings,team,versus,venue,date,match_type
0,V Kohli,AB de Villiers,229,2,Royal Challengers Bangalore,Gujarat Lions,Bengaluru,2016-05-14,Twenty20
1,V Kohli,AB de Villiers,215,2,Royal Challengers Bangalore,Mumbai Indians,Wankhede,2015-05-10,Twenty20
2,KL Rahul,Q de Kock,210,1,LSG,Kolkata Knight Riders,DY Patil,2022-05-18,Twenty20
3,Shubman Gill,B Sai Sudharsan,210,1,Gujarat Titans,Chennai Super Kings,Ahmedabad,2024-05-10,Twenty20
4,SE Marsh,AC Gilchrist,206,2,Punjab Kings,Royal Challengers Bangalore,Dharamsala,2011-05-17,Twenty20


In [3]:
# check_files.py
import os

def print_file_content(filepath):
    print(f"\n=== Content of {filepath} ===")
    try:
        with open(filepath, 'r') as f:
            content = f.read()
            # Print first 300 characters to verify
            print(content[:300] + "...")
            print(f"File size: {len(content)} bytes")
    except Exception as e:
        print(f"Error reading file: {e}")

# Check model_trainer.py
print_file_content("model_trainer.py")

# List all files in directory
print("\n=== Files in current directory ===")
for file in os.listdir("."):
    print(file)


=== Content of model_trainer.py ===
# data_processor.py
import pandas as pd
from pathlib import Path
import logging
from typing import Optional, Dict
import numpy as np

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[logging.FileHandler('data_processing.log'), logging...
File size: 6979 bytes

=== Files in current directory ===
chromedriver.exe
cleaner.py
cleaning.ipynb
clean_data.py
clean_training_data.csv
Data
data_processing.log
data_processor.py
dream11_optimizer.log
fantasy_model.pkl
feature_engineer.py
fixed_team_mapping.py
ipl_2025_full_schedule.csv
ipl_schedule.py
live_data.py
main.py
merged_training_data.csv
merge_data.py
Miniconda3-latest-Linux-x86_64.sh
model_trainer.py
model_training.log
model_training.py
other-feature.py
partnership.py
pipeline_quality_report.csv
team_optimizer.py
test_import.py
__pycache__


In [4]:
# Cell 1: Imports
import pandas as pd
from pathlib import Path
import logging
from typing import Optional
import numpy as np

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[logging.FileHandler('data_processing.log'), logging.StreamHandler()]
)


In [6]:
# Cell 2: Class Definition
class DataProcessor:
    def __init__(self):
        self.data_dirs = {
            'features': Path("Data/features"),
            'matchup': Path("Data/matchup"),
            'partnerships': Path("Data/partnerships_clean.csv"),
            'merged': Path("Data/merged")
        }
        self._validate_directories()

    def _validate_directories(self):
        for path in self.data_dirs.values():
            if path.suffix == '':
                path.mkdir(parents=True, exist_ok=True)

    def _load_with_fallback(self, path: Path, required_cols: list = None) -> pd.DataFrame:
        try:
            if not path.exists():
                logging.warning(f"File not found: {path}")
                return pd.DataFrame(columns=required_cols if required_cols else [])
            df = pd.read_csv(path)
            if required_cols:
                missing = [col for col in required_cols if col not in df.columns]
                if missing:
                    logging.warning(f"Missing columns in {path.name}: {missing}")
                    return pd.DataFrame(columns=required_cols)
            return df
        except Exception as e:
            logging.error(f"Error reading {path}: {str(e)}")
            return pd.DataFrame(columns=required_cols if required_cols else [])

    def _process_matchups(self, players: pd.DataFrame) -> pd.DataFrame:
        for flag in ['hot_batter', 'cold_batter', 'hot_bowler', 'cold_bowler']:
            file = self.data_dirs['matchup'] / f"{flag}s.csv"
            matchup_df = self._load_with_fallback(file, ['Name', 'Team'])
            players[f'is_{flag}'] = players['player'].isin(
                matchup_df['Name'].str.lower().str.strip()
            ).astype(int) if not matchup_df.empty else 0
        return players

    def load_match_data(self, match_num: int) -> Optional[pd.DataFrame]:
        try:
            players = self._load_with_fallback(
                self.data_dirs['features'] / "player_performance_with_features.csv"
            )
            if players.empty:
                raise ValueError("No player data loaded")

            if 'player_type' in players.columns and 'Player Type' not in players.columns:
                players.rename(columns={'player_type': 'Player Type'}, inplace=True)

            players['player'] = players['player_name'].str.lower().str.strip()
            players['team'] = players['team'].str.upper().str.strip()

            if 'Credits' not in players.columns:
                logging.warning("Adding missing 'Credits' column...")
                credit_map = {'BAT': 9.0, 'BOWL': 8.5, 'WK': 8.5, 'AR': 9.0, 'ALL': 9.0}
                players['Credits'] = players['Player Type'].map(credit_map).fillna(8.5)

            must_have = ['batting_ipl_runs', 'batting_ipl_4s', 'batting_ipl_6s',
                         'batting_ipl_avg', 'bowling_ipl_wkts', 'bowling_ipl_econ']
            for col in must_have:
                if col not in players.columns:
                    players[col] = 0.0
                    logging.warning(f"Column '{col}' not found, filled with 0.0")

            players = self._process_matchups(players)

            players['fantasy_points'] = (
                players['batting_ipl_runs'] +
                players['batting_ipl_4s'] * 0.5 +
                players['batting_ipl_6s'] * 1 +
                players['bowling_ipl_wkts'] * 25
            )

            players['boundary_ratio'] = (
                (players['batting_ipl_4s'] + players['batting_ipl_6s']) /
                players['batting_ipl_runs'].replace(0, 1)
            )

            partnerships = self._load_with_fallback(
                self.data_dirs['partnerships'],
                ['player1', 'player2', 'runs']
            )
            if not partnerships.empty:
                partnerships['player1'] = partnerships['player1'].str.lower().str.strip()
                partnerships['player2'] = partnerships['player2'].str.lower().str.strip()
                def get_avg_partnership(player):
                    mask = (partnerships['player1'] == player) | (partnerships['player2'] == player)
                    return partnerships[mask]['runs'].mean() if not partnerships[mask].empty else 0
                players['partnership_avg'] = players['player'].apply(get_avg_partnership)
            else:
                logging.warning("Partnership file empty or missing.")
                players['partnership_avg'] = 0.0

            if len(players) >= 3:
                players['recent_form'] = players['fantasy_points'].rolling(window=3, min_periods=1).mean()
            else:
                players['recent_form'] = players['fantasy_points']

            players.fillna(0, inplace=True)

            for feature in ['boundary_ratio', 'partnership_avg', 'recent_form']:
                if feature not in players.columns:
                    logging.error(f"❌ Missing feature '{feature}' before training!")
                    players[feature] = 0.0

            # 🔹 Add 'blom' column here
            players['blom'] = 'nf'

            merged_file = self.data_dirs['merged'] / f"merged_match_{match_num}.csv"
            players.to_csv(merged_file, index=False)
            logging.info(f"✅ Saved merged data: {merged_file}")
            return players

        except Exception as e:
            logging.error(f"❌ Failed to process match {match_num}: {str(e)}")
            return None
